In [1]:
import pennylane as qml
import autograd.numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import math
import pennylane as qml
from pennylane.operation import Operation
from pennylane import numpy as np

# number of qubits and define quantum device
n_qubit = 4
dev = qml.device('default.mixed', wires=n_qubit)
dev.operations.add("RXX")

# define gradient recipe
INV_SQRT2 = 1 / math.sqrt(2)
c1 = INV_SQRT2 * (np.sqrt(2) + 1) / 4
c2 = INV_SQRT2 * (np.sqrt(2) - 1) / 4
a = np.pi / 2
b = 3 * np.pi / 2
four_term_grad_recipe = ([[c1, 1, a], [-c1, 1, -a], [-c2, 1, b], [c2, 1, -b]],)

# define what kind of noise
noise_encoded_list = ['bitflip', 'depolarizing_channel', 'amplitude_damping']

# define parameters
params_encoded = np.array([1.1, 0.7, -0.6]) # for dephasing

params = np.array([[[ 0.99428273,  1.09710887, -0.56566131, -0.06691773],
         [ 1.77189978, -0.36773718,  0.69736815, -1.21426263],
         [ 0.25244028,  0.33687556, -1.33784468,  2.21047394],
         [ 0.3394889 , -1.05655419, -0.7571113 , -1.77781283]],

        [[-0.84773032,  1.83898219,  0.31333539,  0.70865689],
         [-1.02512602, -0.68341346, -0.58727157,  0.24246219],
         [-0.07084525, -0.76039399, -0.21280071, -0.21102259],
         [-0.88367094, -0.16177464, -0.83400285,  0.61017164]],

        [[ 0.53281143, -0.95501388,  2.38233895, -0.29837746],
         [-1.06704017,  0.24286941,  0.73346593,  1.12401983],
         [-0.25551071, -2.03569606, -0.19236259, -0.67839622],
         [ 0.39020518, -0.10392657, -0.05518679, -0.35891302]],

        [[ 0.12817979, -0.00992391, -0.00364549, -1.40676109],
         [ 0.05122566,  0.00691019,  0.42308062,  0.99433363],
         [ 1.48833531, -0.33839754, -2.0910268 ,  0.53049627],
         [-0.8858284 ,  0.09423672, -1.82323287, -0.41916103]],

        [[-0.55314528,  0.36325934, -0.31057847,  0.13886429],
         [-0.33743291,  0.07965355,  1.20437349,  0.23126415],
         [ 2.50047872,  1.12006902,  0.87573478, -0.43730202],
         [ 1.41811439,  0.316319  ,  0.61662017,  0.94854172]],

        [[ 1.30664094, -0.48438619,  0.43446318, -0.74382484],
         [ 0.163874  , -2.89143975,  1.22669873,  0.03160277],
         [ 1.01211061, -0.96099056, -2.15876514, -0.27013816],
         [ 1.8390521 ,  0.12355094,  1.2428606 , -0.53097397]]]) # for optimizing

# define RXX gate
class RXX(Operation):
    num_params = 1
    num_wires = 2
    par_domain = "R"

    grad_method = "A"
    grad_recipe = four_term_grad_recipe # This is the default but we write it down explicitly here.

    generator = [(qml.PauliX(0) @ qml.PauliX(1)).matrix, -0.5]

    @staticmethod
    def decomposition(theta, wires):
        return [qml.PauliRot(theta, 'XX', wires=wires)]

    @staticmethod
    def _matrix(*params):
        theta = params[0]
        c = np.cos(0.5 * theta)
        s = np.sin(0.5 * theta)
        return np.array(
            [
                [c, 0, 0, -s],
                [0, c, -s, 0],
                [0, -s, c, 0],
                [-s, 0, 0, c]
            ]
        )

    def adjoint(self):
        return RXX(-self.data[0], wires=self.wires)

# squeezing part
def squeezing(params):
    
    n_qubit = len(params[0])
    
    # RY gate
    for i in range(n_qubit):
        qml.RY(np.pi/2, wires=i)
    
    # GMS_z gate
    for i in range(n_qubit):
        for j in range(i+1, n_qubit):
            qml.MultiRZ(params[0][i][j], wires=[i, j])
    
    # RX gate
    for i in range(n_qubit):
        qml.RX(params[1][0][i], wires=i)
    
    # GMS_x gate
    for i in range(n_qubit):
        for j in range(i+1, n_qubit):
            RXX(params[2][i][j], wires=[i, j])

# squeezing circuit
@qml.qnode(dev)
def multiple_phase_sensing(params, params_encoded, p, noise_encoded):
    
    # Squeezing
    squeezing(params[:3])
    
    for i in range(4):
        qml.RZ(-params_encoded[2], wires=i)
        qml.RY(-params_encoded[1], wires=i)
        qml.RZ(params_encoded[0], wires=i)
        qml.RY(params_encoded[1], wires=i)
        qml.RZ(params_encoded[2], wires=i)
    
    # Noise:
    if noise_encoded == 'bitflip':
        for i in range(4):
            qml.BitFlip(p, wires=i)
    elif noise_encoded == 'depolarizing_channel':
        for i in range(4):
            qml.DepolarizingChannel(p, wires=i)
    elif noise_encoded == 'amplitude_damping':
        for i in range(4):
            qml.AmplitudeDamping(p, wires=i)
    
    # Adjoint of squeezing
    qml.adjoint(squeezing)(params[3:])
    
    # Measurement 
    return qml.density_matrix(wires=range(len(params[0])))

# shiff rule of fisher information
def fisher_shift_rule(i, j, params, params_encoded, shift, p, noise_encoded):
    
    rho = multiple_phase_sensing(params, params_encoded, p, noise_encoded)
    product_1 = 2*np.linalg.inv(np.kron(np.conj(rho), np.identity(4*4)) + np.kron(np.identity(4*4), rho))
    
    params_encoded_i_plus = params_encoded.copy()
    params_encoded_i_subs = params_encoded.copy()
    params_encoded_j_plus = params_encoded.copy()
    params_encoded_j_subs = params_encoded.copy()

    params_encoded_i_plus[i] = params_encoded_i_plus[i] + shift
    params_encoded_i_subs[i] = params_encoded_i_subs[i] - shift
    params_encoded_j_plus[j] = params_encoded_j_plus[j] + shift
    params_encoded_j_subs[j] = params_encoded_j_subs[j] - shift

    diff_i = (multiple_phase_sensing(params, params_encoded_i_plus, p, noise_encoded) - multiple_phase_sensing(params, params_encoded_i_subs, p, noise_encoded))/(2*np.sin(shift))
    diff_j = (multiple_phase_sensing(params, params_encoded_j_plus, p, noise_encoded) - multiple_phase_sensing(params, params_encoded_j_subs, p, noise_encoded))/(2*np.sin(shift))
        
    diff_i = diff_i.flatten('F')
    diff_i.shape = (4**4, 1)
    diff_i = diff_i.conj().T
    
    diff_j = diff_j.flatten('F')
    diff_j.shape = (4**4, 1)
    
    product = np.matmul(diff_i, product_1)
    product = np.matmul(product, diff_j)
        
    return product[0][0]

# calculate fisher matrix
def fisher_matrix(params, params_encoded, shift, p, noise_encoded):
    n = len(params_encoded)
    results = [[fisher_shift_rule(i, j, params, params_encoded, shift, p, noise_encoded) for j in range(n)] for i in range(n)]
    return np.array(results)

# cost function
def cost(params):
    
    global params_encoded, noise_encoded, p_noise
    
    # Noise: ps = [0.001, 0.01, 0.1, 0.2]
    arr = fisher_matrix(params, params_encoded, np.pi/20, p_noise, noise_encoded)
    
    return np.trace(np.linalg.inv(arr))

In [2]:
a = multiple_phase_sensing(params, params_encoded, 0, noise_encoded_list[0])

In [7]:
noise_encoded = noise_encoded_list[0]
p_noise = 0
params = np.random.normal(size=(6, n_qubit, n_qubit))

cost(params)

(2.8479422215861476e-13-4.3518036586678976e-13j)

In [59]:
params_opt = params.copy()

# Define optimizer
opt = qml.AdamOptimizer()
steps = 3
        
for i in range(steps):
    params_opt = opt.step(cost, params_opt)

TypeError: Can't differentiate w.r.t. type <class 'int'>

In [8]:
# results
results = {}

# Define optimizer
opt = qml.AdamOptimizer()
steps = 50

for noise in noise_encoded_list:

    costs_array = []
    
    for p in np.arange(0, 0.21, 0.01):
        
        noise_encoded = noise
        p_noise = p
        params_opt = params.copy()
        
#         for i in range(steps):
#             params_opt = opt.step(cost, params_opt)

        costs_array.append(cost(params_opt))
    
        results[noise_encoded] = costs_array
        
# pd.DataFrame(results).to_csv('classical_bound_with_noise.csv')

In [9]:
pd.DataFrame(results)

,bitflip,depolarizing_channel,amplitude_damping
0,2.847942e-13-4.351804e-13j,2.847942e-13-4.351804e-13j,2.847942e-13-4.351804e-13j
1,1.355969e+01+2.314257e-08j,1.088709e+02+6.764820e-10j,6.063954e+00-5.634288e-03j
2,3.402348e+01+2.227051e-07j,1.241441e+02-2.874335e-10j,1.154615e+01-4.938790e-03j
3,6.333323e+01-2.421107e-08j,1.344464e+02-1.695361e-11j,1.340431e+01-3.699376e-04j
4,8.649520e+01-1.041328e-08j,1.438279e+02-1.004763e-11j,1.503060e+01-3.422844e-05j
5,1.011400e+02-4.956243e-09j,1.532476e+02-2.747479e-12j,1.715065e+01-1.643879e-06j
6,1.102247e+02-9.627036e-11j,1.630847e+02-3.033487e-11j,1.999262e+01-5.466805e-05j
7,1.161884e+02+1.517002e-11j,1.735503e+02-1.538436e-11j,2.363389e+01-2.076930e-05j
8,1.204177e+02+1.103302e-10j,1.847978e+02+2.028546e-11j,2.806519e+01+1.104424e-05j
9,1.236556e+02-5.123777e-11j,1.969604e+02-4.163567e-12j,3.321990e+01-8.242554e-06j


In [ ]:
for i in range(steps):
    params = opt.step(cost, params)

In [3]:
qml.version()

'0.17.0'